In [49]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split, GridSearchCV
from skimage import io, transform
print(os.listdir("../input/pollendataset/PollenDataset/"))

['Read.ipynb', 'images', 'Read-skimage.ipynb', 'README.md', 'pollen_data.csv']


In [50]:
from keras.models import Sequential
from keras.layers import Conv2D, Dropout, MaxPool2D, Flatten, Dense
from keras.optimizers import SGD, RMSprop
from keras.wrappers.scikit_learn import KerasClassifier

In [51]:
data = pd.read_csv("../input/pollendataset/PollenDataset/pollen_data.csv")

In [52]:
data.head(10)

,Unnamed: 0,filename,pollen_carrying
0,0,P10057-125r.jpg,1
1,1,P57380-120r.jpg,1
2,2,NP63671-221r.jpg,0
3,3,NP53776-47r.jpg,0
4,4,NP13382-201r.jpg,0
5,5,P63866-224r.jpg,1
6,6,P22071-82r.jpg,1
7,7,NP64043-227r.jpg,0
8,8,NP59420-165r.jpg,0
9,9,P63758-222r.jpg,1


In [53]:
def dataset_gen(data, size = (300,180)):
    
    img_data = []
    labels = []
    for img_name, pollen_carrying in zip(data['filename'], data['pollen_carrying']):
        img = io.imread(os.path.join("../input/pollendataset/PollenDataset/images", img_name))
        img = transform.resize(img, size, mode = 'constant')
        img_data.append(img)
        labels.append(pollen_carrying)
        
    return np.array(img_data), np.array(labels)

In [54]:
x, y = dataset_gen(data)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 7)

y_train = tf.keras.utils.to_categorical(y_train, num_classes = 2)
y_test = tf.keras.utils.to_categorical(y_test, num_classes = 2)

x_train_val, x_test_val, y_train_val, y_test_val = train_test_split(x_train, y_train, test_size = 0.15, random_state = 7)

for data in [ x_train,y_train,x_test,y_test,x_train_val,x_test_val,y_train_val,y_test_val]:
    print(data.shape)

(571, 300, 180, 3)
(571, 2)
(143, 300, 180, 3)
(143, 2)
(485, 300, 180, 3)
(86, 300, 180, 3)
(485, 2)
(86, 2)


In [59]:
model = Sequential()
model.add(Conv2D(32, (3,3), input_shape = (300,180,3), activation = 'relu'))
model.add(MaxPool2D((2,2)))
model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(MaxPool2D((2,2)))
model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(MaxPool2D((2,2)))
model.add(Conv2D(64,
                 (3,3),
                activation = 'relu'))
model.add(MaxPool2D((2,2)))
model.add(MaxPool2D((2,2)))
model.add(Flatten())
model.add(Dropout(0.60))
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.50))
model.add(Dense(2, activation = 'softmax'))
model.summary()

model.compile(optimizer = 'Adadelta', loss = 'categorical_crossentropy', metrics = ['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_29 (Conv2D)           (None, 298, 178, 32)      896       
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 149, 89, 32)       0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 147, 87, 64)       18496     
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 73, 43, 64)        0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 71, 41, 64)        36928     
_________________________________________________________________
max_pooling2d_38 (MaxPooling (None, 35, 20, 64)        0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 33, 18, 64)        36928     
__________

In [60]:
model.fit(x = x_train_val,
         y = y_train_val,
         batch_size = 16,
         epochs = 50,
         validation_data = (x_test_val, y_test_val))

Train on 485 samples, validate on 86 samples
Epoch 1/50
485/485 [==============================] - 3s 7ms/step - loss: 0.7001 - acc: 0.5505 - val_loss: 0.6902 - val_acc: 0.5000
Epoch 2/50
485/485 [==============================] - 2s 5ms/step - loss: 0.6877 - acc: 0.5773 - val_loss: 0.6389 - val_acc: 0.5233
Epoch 3/50
485/485 [==============================] - 2s 5ms/step - loss: 0.6289 - acc: 0.6866 - val_loss: 0.6877 - val_acc: 0.6047
Epoch 4/50
485/485 [==============================] - 2s 5ms/step - loss: 0.5868 - acc: 0.6948 - val_loss: 0.4353 - val_acc: 0.8605
Epoch 5/50
485/485 [==============================] - 2s 5ms/step - loss: 0.4669 - acc: 0.7938 - val_loss: 0.4149 - val_acc: 0.8256
Epoch 6/50
485/485 [==============================] - 2s 5ms/step - loss: 0.4386 - acc: 0.8144 - val_loss: 0.6418 - val_acc: 0.6163
Epoch 7/50
485/485 [==============================] - 2s 5ms/step - loss: 0.3477 - acc: 0.8536 - val_loss: 0.4752 - val_acc: 0.7907
Epoch 8/50
485/485 [===========

In [61]:
pred = model.evaluate(x_test,
                      y_test,
                    batch_size = 32)

143/143 [==============================] - 0s 3ms/step


In [62]:
print(pred)

[0.12112641127102754, 0.972027972027972]
